Odredi vrednost automobila koji trosi 9l i kvari se 8 puta na 100 000km. \\
Vrednost moze biti: niska, srednja, visoka. \\
Potrosnja: mala, srednja, visoka \\
Pouzdanost: niska, visoka

In [2]:
class FuzzyInput():
    # xs, ys - kljucne tacke
    # x0 - broj sa ulaza
    def __init__(self, name, xs, ys, x0):
        self.name = name
        self.points = list(zip(xs, ys))
        self.mi = self.calculateMi(x0)

    def calculateMi(self, x0):
        if x0 < self.points[0][0]: # levo od prve tacke
            return self.points[0][1]
        if x0 > self.points[-1][0]: # desno od poslednje
            return self.points[-1][1]

        for i in range(0, len(self.points)-1):
            x1 = self.points[i][0]
            x2 = self.points[i+1][0]

            if x0 >= x1 and x0 < x2:
                y1 = self.points[i][1]
                y2 = self.points[i+1][1]
                if y1 == y2:
                    return y1
                if y1 < y2: # rastuca funkcija
                    return (x0 - x1) / (x2 - x1)
                else: # opadajuca funkcija
                    return (x2 - x0) / (x2 - x1)

In [3]:
fi = FuzzyInput('mala', [3, 10], [1, 0], 9)
fi.mi

0.14285714285714285

In [4]:
class FuzzyOutput():
    def __init__(self, name, xs, ys):
        self.name = name
        self.points = list(zip(xs, ys))
        self.mi = 0
        avg = 0
        n = 0

        for p in self.points:
            if p[1] == 1: # y == 1 
                avg += p[0]
                n += 1

        self.c = avg / n

In [5]:
fo = FuzzyOutput('srednja', [1, 15, 25, 40], [0, 1, 1, 0])
fo.c

20.0

In [6]:
from enum import Enum, unique

@unique
class LogicOP(Enum):
    AND = 0
    OR = 1

class Rules():
    def __init__(self, input1, input2, output, operator):
        self.input1 = input1
        self.input2 = input2
        self.output = output

        if operator == LogicOP.AND:
            self.output.mi = max(self.output.mi, min(self.input1.mi, self.input2.mi))
        else:
            self.output.mi = max(self.output.mi, max(self.input1.mi, self.input2.mi))

In [10]:
potrosnja = []
potrosnja.append(FuzzyInput('mala', [3, 10], [1, 0], 9))
potrosnja.append(FuzzyInput('srednja', [7, 10, 12, 15], [0, 1, 1, 0], 9))
potrosnja.append(FuzzyInput('velika', [12, 15], [0, 1], 9))

pouzdanost = []
pouzdanost.append(FuzzyInput('visoka', [5, 10], [1, 0], 8))
pouzdanost.append(FuzzyInput('niska', [8, 15], [0, 1], 8))

vrednost = []
vrednost.append(FuzzyOutput('mala', [7, 15], [1, 0]))
vrednost.append(FuzzyOutput('srednja', [7, 15, 25, 40], [0, 1, 1, 0]))
vrednost.append(FuzzyOutput('velika', [25, 40], [0, 1]))

rules = []
rules.append(Rules(potrosnja[0], pouzdanost[0], vrednost[2], LogicOP.AND))
rules.append(Rules(potrosnja[0], pouzdanost[1], vrednost[1], LogicOP.AND))
rules.append(Rules(potrosnja[1], pouzdanost[0], vrednost[1], LogicOP.AND))
rules.append(Rules(potrosnja[1], pouzdanost[1], vrednost[1], LogicOP.AND))
rules.append(Rules(potrosnja[2], pouzdanost[0], vrednost[1], LogicOP.AND))
rules.append(Rules(potrosnja[2], pouzdanost[1], vrednost[0], LogicOP.AND))

In [11]:
# Defazifikacija
brojioc = 0
imenioc = 0

for v in vrednost:
    brojioc += v.mi * v.c
    imenioc += v.mi

rez = brojioc / imenioc
rez

25.26315789473684